In [11]:
import sys
import os
from dotenv import load_dotenv

sys.path.append(os.path.abspath(".."))

load_dotenv("../.env")

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
from app.rag.repository.meili import LangChainMeiliRepository
from app.core.config import settings

repo = LangChainMeiliRepository()

import meilisearch

client = meilisearch.Client(settings.MEILI_HTTP_ADDR, settings.MEILI_KEY)

index_name = "langchain-demo" 
index = client.index(index_name)

index.update_embedders({
    "default": {
        "source": "userProvided",
        "dimensions": 1536
    }
})

TaskInfo(task_uid=256, index_uid='langchain-demo', status='enqueued', type='settingsUpdate', enqueued_at=datetime.datetime(2025, 12, 28, 16, 34, 20, 677674))

In [13]:
# 테스트용 원본 문서 데이터
test_docs = [
    "연차 사용 규정: 연차는 최소 1일 전에 신청해야 하며, 팀장의 승인이 필요하다.",
    "재택 근무 규정: 주 2회 재택이 가능하며, 코어 타임(1시~4시)에는 메신저 응답이 필수다.",
    "식대 지원: 점심 식대는 13,000원까지 법인카드로 지원된다.",
    "서강대 맛집: 학교 앞 '마포불백'이 가성비가 좋다."
]

# 메타데이터
metadatas = [
    {"category": "HR", "role": "member"},
    {"category": "Work", "role": "member"},
    {"category": "Welfare", "role": "member"},
    {"category": "Food", "role": "all"}
]

# 데이터 넣기
ids = repo.vector_store.add_texts(texts=test_docs, metadatas=metadatas)
print(f"테스트 데이터 {len(ids)}개 적재 완료")

테스트 데이터 4개 적재 완료


In [14]:
# [Cell 4] 검색 실험
query = "밥값 최소 얼마 지원돼?"

# k=2 (상위 2개)
results = repo.retrieve(query=query, k=2)

print(f"질문: {query}\n")

print(f"top 2 docs")
for i, doc in enumerate(results):
    print(f"[{i+1}등] {doc}")

질문: 밥값 최소 얼마 지원돼?

top 2 docs
[1등] page_content='식대 지원: 점심 식대는 13,000원까지 법인카드로 지원된다.' metadata={'category': 'Welfare', 'role': 'member'}
[2등] page_content='식대 지원: 점심 식대는 13,000원까지 법인카드로 지원된다.' metadata={'category': 'Welfare', 'role': 'member'}


In [15]:
from app.rag.service.llm import LlmService

llm_service = LlmService()
print(await llm_service.generate_answer(query=query, context=results, role="member", session_id="test-user-1"))


answer='밥값은 최소 13,000원까지 법인카드로 지원됩니다.' source=['Unknown']
